In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os

import VMF

GPUs are used!
Classes loaded
0


/uio/hume/student-u44/lmsunde/projects/BNN/AliaksandrFolder/VMF.py:274: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v0 = torch.cat([v0, torch.tensor(w0[det >= 0]).to(DEVICE)])
/opt/uio/modules/rhel8/easybuild/software/Miniconda3/lmsunde/envs/BNN2/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


1
tensor(-44875.5742, device='cuda:1', grad_fn=<AddBackward0>)
tensor(161.9264, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
2
tensor(-44877.1875, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.4850, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
3
tensor(-44879.4414, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.3879, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
4
tensor(-44877.4297, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.2970, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
5
tensor(-44878.0703, device='cuda:1', grad_fn=<AddBackward0>)
tensor(159.8449, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
6
tensor(-44878.8008, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.1437, device='cu

46
tensor(-44895.5234, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.6345, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
47
tensor(-44894.1719, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.6839, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
48
tensor(-44894.4062, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.4504, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
49
tensor(-44895.8320, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.1073, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
50
tensor(-44895.8672, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.7605, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
51
tensor(-44896.8789, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.0521, devi

91
tensor(-44912.7422, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.1607, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
92
tensor(-44913.2070, device='cuda:1', grad_fn=<AddBackward0>)
tensor(159.9827, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
93
tensor(-44911.2109, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.3868, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
94
tensor(-44912.2031, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.2595, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
95
tensor(-44913.4492, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.5222, device='cuda:1', grad_fn=<NllLossBackward0>)
tensor(0.0001, device='cuda:1', grad_fn=<MeanBackward0>)
96
tensor(-44912.9727, device='cuda:1', grad_fn=<AddBackward0>)
tensor(160.1829, devi

In [ ]:
#epochs = 225
#trtimes  = np.zeros(epochs)
# make inference on 10 networks
#for i in range(0, 1):
#    print(i)
#    torch.manual_seed(i)
#    net = VMF.BayesianNetwork().to(VMF.DEVICE)
#    optimizer = optim.Adam(net.parameters(), lr=0.0001)
#    for epoch in range(epochs):
#
#        trtimes[epoch] = VMF.train(net, optimizer, epoch, i)
#        print(net.l1.weight_mu.mean())
#
#    res = VMF.test_ensemble()
#
#    np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")
r"""
layers = [(256, 400),(400, 600),(400, 600),(600, 5)]
self_layerdict = {}
i = 0
while i<len(layers):
            name = 'self_l'+str(i)
            self_layerdict[name] = layers[i]
            i = i+1
            print(name)
for k,v in self_layerdict.items():
    exec("%s = %s" % (k, v))
self_l1
"""
torch.set_printoptions(edgeitems=1)
x=torch.tensor((4,4))
x

In [3]:
torch.__version__


'1.13.0.post200'